# [Exciton-Phonon Tutorial](#exc-phonontutorial)
Exciton-phonon coupling refers to the interaction between excitons (bound states of an electron and a hole) and phonons (quanta of lattice vibrations) in a material. This coupling can significantly affect the optical and electronic properties of the material. When an exciton interacts with phonons, it can scatter, leading to changes in its energy and momentum. This process is crucial for understanding phenomena such as thermalization, relaxation, and recombination of excitons in semiconductors and other materials.
This workflow is split up into several sections:

0. [DFT](#step-0-dft-ground-state-calculation-using-pwx)
1. [DFPT](#step-1-dfpt-calculation-using-phx)
2. [ELPH](#step-2-dvscf-calculation-using-letzelph-yambopy)
Compute the electron-phonon matrix elements $g_{kkp}$ using the [LetzElPhC code](https://github.com/muralidhar-nalabothula/LetzElPhC/). 
3. [MBPT](#step-3-mbpt-calculation-using-yambo)
Compute BSE kernel and diagonalize on top of GW calculation, using the Yambo code to get the exciton wavefunctions.
4. [EXC-PH](#step-4-exciton-phonon-calculation-using-yambopy)
Combine all of the above to compute the exciton-phonon matrix elements that are used to compute photoluminesensce spectrum with python [scripts](https://github.com/muralidhar-nalabothula/PhdScripts/).

We will go through these steps with a simple system of hBN monolayer that can be readily run with relatively small computational cost. Step 1 of calculating the phonons takes some time, but one can parallelize and calculate at the same time the GW-BSE kernel at step 3.
For more detailed information on exciton-phonon coupling and its effects on luminescence in hexagonal boron nitride, you can refer to the paper: {cite}`PhysRevMaterials.7.024006`.

**NB**: For a detailed tutorials on Quantum Espresso we refer to this [Webpage](https://pranabdas.github.io/espresso/setup/jupyter/)

## Step 0: [DFT](#step-0-dft-ground-state-calculation-using-pwx)
1. Run a self-consistent field (scf) calculation with `pw.x`. We suggest creating a new directory called `mkdir scf` in your ROOT FOLDER. Inside `scf` create a pw.x input file `touch scf.in`
<details>
  <summary><strong>▶️ Expand for `scf` `pw.x` input file</strong></summary>

  <pre><code class="language-bash">
&control
  calculation = 'scf'
  outdir = '.'
  prefix = 'hBN'
  pseudo_dir = '{PATH_TO_PSEUDOS}' # Change me: We used pseudos from PseudoDojo SR_v0.5 LDA_Standard
  verbosity = 'high'
  nstep = 20000
/&end
&system
  ecutwfc = 120
  ibrav = 0
  nat = 4
  ntyp = 2
  occupations = 'fixed'
  force_symmorphic = .true.
/&end
&electrons
            conv_thr = 1e-15
/&end
&IONS
/
&cell
  cell_dynamics = "bfgs"
/
ATOMIC_SPECIES
B       10.811   B.upf
N       14.0067  N.upf
CELL_PARAMETERS (angstrom)
   2.489045057  -0.000000000   0.000000000
  -1.244522529   2.155576251  -0.000000000
   0.000000000  -0.000000000   6.482867531
ATOMIC_POSITIONS (crystal)
B                0.6666666667        0.3333333333        0.5000000000
B                0.3333333333        0.6666666667        0.0000000000
N                0.6666666667        0.3333333333       -0.0000000000
N                0.3333333333        0.6666666667        0.5000000000
K_POINTS automatic
12 12 4 0 0 0
</code></pre>
</details>


2. Run a non self-consistent field (nscf) calculation with `pw.x`. Create a new directory in ROOT FOLDER `mkdir nscf_6x6x1` and link or copy the `ln -s ../scf/hBN.save ./` folder to `nscf`. For the purpose of the tutorial we use a non-converged 6x6x1 grid (**NB** grid in the nscf has to be the same as the one used in [DFPT](#step-1-dfpt)):
<details>
  <summary><strong>▶️ Expand for `nscf` pw.x input file</strong></summary>

  <pre><code class="language-bash">
&control
  calculation = 'nscf'
  outdir = '.'
  prefix = 'hBN'
  pseudo_dir = '{PATH_TO_PSEUDOS}' # Change me: We used pseudos from PseudoDojo SR_v0.5 LDA_Standard
  verbosity = 'high'
  nstep = 20000
/&end
&system
  ecutwfc = 120
  ibrav = 0
  nat = 4
  ntyp = 2
  occupations = 'fixed'
  force_symmorphic = .true.
/&end
&electrons
            conv_thr = 1e-15
/&end
&IONS
/
&cell
  cell_dynamics = "bfgs"
/
ATOMIC_SPECIES
B       10.811   B.upf
N       14.0067  N.upf
CELL_PARAMETERS (angstrom)
   2.489045057  -0.000000000   0.000000000
  -1.244522529   2.155576251  -0.000000000
   0.000000000  -0.000000000   6.482867531
ATOMIC_POSITIONS (crystal)
B                0.6666666667        0.3333333333        0.5000000000
B                0.3333333333        0.6666666667        0.0000000000
N                0.6666666667        0.3333333333       -0.0000000000
N                0.3333333333        0.6666666667        0.5000000000
K_POINTS automatic
12 12 1 0 0 0
</code></pre>
</details>

## Step 1: [DFPT](#step-1-dfpt-calculation-using-phx)

The goal of this step is to compute the electron-phonon matrix elements $g_{q\nu}(\mathbf{k}, i, j)$ defined as 

$g_{\mathbf{q} \nu}(\mathbf{k}, i, j)=\left(\frac{\hbar}{2 M \omega_{\mathbf{q} \nu}}\right)^{1 / 2}\left\langle\psi_{i, \mathbf{k}}\right| \frac{\partial V_{S C F}}{\partial \hat{u}_{\mathbf{q} \nu}} \cdot \hat{\epsilon}_{\mathbf{q} \nu}\left|\psi_{j, \mathbf{k}+\mathbf{q}}\right\rangle$

($\frac{\partial V_{S C F}}{\partial \hat{u}_{\mathbf{q} \nu}}$ is what we call `dvscf`)

and the dynamical matrices defined as:

$D_{s t}^{\alpha \beta}(\mathbf{q})=\frac{1}{\sqrt{M_s M_t}} \sum_{\mathbf{R}} C_{s t}^{\alpha \beta}(\mathbf{R}) \exp (i \mathbf{q} \cdot \mathbf{R})$

where $C_{I J}^{\alpha \beta}(\mathbf{R})$ is the matrix of inter-atomic force constant (IFC), defined as the second derivative with respect to the total energy $E(\mathbf{R})$

$C_{I J}^{\alpha \beta} \equiv \frac{\partial^2 E(\{\mathbf{R}\})}{\partial R_I^\alpha \partial R_J^\beta}$

$M_{s/t}$: mass of the ions

To compute the el-ph matrix elements we make use of the library [LetzElPhC](../external_files/LetzElPhC_documentation.pdf)

**Before** Computing $g_{q\nu}(\mathbf{k}, i, j)$ and $D_{s t}^{\alpha \beta}(\mathbf{q})$  we need the:
- Kohn-Sham wavefunctions (obtained in [Step 0](#step-0-dft))
- phonon eigenvectors and perturbed Hartree ($\Delta V_{Hartree}$) and Exchange potentials ($\Delta V_{exchange}$) due to the phonon mode (obtained via `DFPT`)

The phonon eigenvectors are obtained via a **DFPT** calculation (`ph.x < ph.in > log_ph.out`). We work in a fresh folder in ROOT FOLDER `mkdir phonons`:
<details>
    <summary><strong>▶️ Expand for `dfpt` `ph.x` input file</strong></summary>
    <pre><code class="language-bash">
phonons on a grid
 &inputph
  reduce_io = .true.
  nq1 = 6,
  nq2 = 6,
  nq3 = 1,
  fildyn = 'hBN.dyn.xml',
  tr2_ph = 1e-17,
  prefix = 'hBN',
  trans = .true.,
  electron_phonon = 'dvscf', # compute 
  fildvscf = 'dvscf',
  ldisp = .true.,
  recover = .true.
/
</code></pre>
</details>

Note that if you are submitting jobs to the cluster it might be optimal to have a customized parallelization scheme within `ph.x` and specify your parallization scheme over images, k-points and plane-waves.

<details>
    <summary><strong>▶️ Expand for slurm submission job script</strong></summary>
    <pre><code class="language-bash">
#!/bin/bash -l
#SBATCH -J "hBN_ph"
#SBATCH -N 1
#SBATCH --ntasks-per-node=32 #mpi parallelization
#SBATCH --cpus-per-task=1 #open mp
#SBATCH --time=6:00:00
#SBATCH --hint=multithread
#SBATCH --qos=normal

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK
export SRUN_CPUS_PER_TASK=$SLURM_CPUS_PER_TASK

export PATH="$PATH:/home/users/rreho/codes/q-e-qe-7.4.1/bin:/home/users/rreho/personalcodes/lumen/bin"

module load env/development/2024a
module load math/ELPA/2024.05.001-foss-2024a
module load data/netCDF-Fortran/4.6.1-gompi-2024a

srun -n ${SLURM_NTASKS} ph.x -ni 4 -nk 2 < ph.in > log_ph.out
</code></pre>
</details>

## Step 2: [ELPH](#step-2-dvscf-calculation-using-letzelph-yambopy)
`LetzElPhC` requires the initialization of the yambo SAVE folder. Go to the `nscf_6x6x1` folder inside `hBN.save` and run `p2y` followed by `yambo`.
For convenience we advise to create in the ROOT_FOLDER a `database` folder where you store the `yambo` `SAVE` folder (`mv nscf_6x6x1/hBN.save/SAVE ./database`).

Navigate in `phonons` and run the preprocess step needed by `lelphc`:

Create a new folder in ROOT_FOLDER `mkdir lelph` and run:

`lelphc -pp --code=qe -F ph.in`

Now, the calculation of the electron-phonon coupling elements can be executed. Run the following command
`lelph -F lelph.in`

<details>
    <summary><strong>▶️ Expand for `lelph` input file</strong></summary>
    <pre><code class="language-bash">
nkpool          = 1
# k point parallelization
nqpool          = 1
# q point parallelization
## note ( nkpool * nqpool ) must divide total number of cpus .
## For example , if you run the code with 12 processess ,
## and set nkpool = 3 and nqpool = 2
## then , we have 2 sets of cpus working subset of qpoints (qpool 1 and qpool 2).
## Each group has 3 sub groups working on
## subset of kpoints . So in total , we have 6 subgroups , each
## having 2 cpus that distribute plane waves
## {1 ,2 ,3 ,4 ,5 ,6 ,7 ,8 ,9 ,10 ,11 ,12} ( total cpus )
## _ _ _ _ _ _ _ _ _ _|_ _ _ _ _ _ _ _ _ _ _ _
# # |      divided in to 2 qpools            |
# # ( qpool 1) {1 ,2 ,3 ,4 ,5 ,6} ( qpool 2) {7 ,8 ,9 ,10 ,11 ,12}
# #           |                              |
## _ _ _ _ _ _|_ _ _ _ _ _ _ _     _ _ __ _ _|_ _ _ _ _ _ _ _
## |          |               |   |         |               |
## kp1       kp2            kp3  kp1      kp2              kp3
## where kp1 are kpools each containg 2
## cpus work on subset of plane waves
start_bnd       = 7
# starting band
end_bnd         = 10
# last band
save_dir        = ../database/SAVE
# save dir
ph_save_dir     = ../phonons/ph_save
#ph_save directory which contains all phonon files 
kernel          = dfpt
convention      = yambo
# standard/yambo, If standard (default) 
# <k+q|dV|k> is computed. if yambo, <k|dV|k-q> is outputed 
###  ##, !, ; are considered as comments
</code></pre>
</details>

## Step 3:GW-BSE [MBPT](#step-3-mbpt-calculation-using-yambo)
Create a `GW_BSE` folder and link the `database/SAVE` folder in this new directory.

Run in sequence `GW` and `BSE` calcutions via the commands

- GW `yambo -F gw.in -J './gw'` 
- BSE `yambo -F bse.in -J './bse,./gw'`
  
<details>
    <summary><strong>▶️ Expand for `GW` input file</strong></summary>
    <pre><code class="language-bash">
# gw input
dyson                           # solver dyson equation 
gw0                              # do a G0W0 calculation (only one iteration)
ppa                              # plasmon pole model to model the frequency depenenent dielectric
HF_and_locXC                     # Hartree fork exchange
em1d                             # compute and use dynamical screening. (
FFTGvecs= 60 Ry                  #  fft grid for computing the exchange
NLogCPUs=1                       # number of cputs to be used to output the log file. always keep it 1
X_and_IO_CPU= "4 1 4 1"        ## parallel scheme 
X_and_IO_ROLEs= "q k c v"        ## parallel scheme 
DIP_CPU= "4 4 1"                ## 
DIP_ROLEs= "k c v"
SE_CPU= "4 4 1"
SE_ROLEs= "q qp b"
EXXRLvcs= 120 Ry
VXCRLvcs= 120 Ry
Chimod= "HARTREE"
RandQpts=10000000                     # [RIM] Number of random q-points in the BZ
RandGvec=400
% BndsRnXp ### bands used to construct dielectric screening (within RPA approximation)
    1 | 20 |
%
NGsBlkXp= 5000 mHa               # Energy cutoff for the Gvectors when computing the screened interaction
% LongDrXp
 1.000000 | 1.000000 | 1.000000 |
%
PPAPntXp= 27.21138         eV
XTermKind= "none"
% GbndRnge
    1 | 20 |
%
GDamping= 0.100000         eV
GTermKind= "BG"
GTermEn= 40.81708          eV
DysSolver= "n"
QPExpand
%QPkrange
1|7|7|10|
%
</code></pre>
</details>

<details>
    <summary><strong>▶️ Expand for `BSE` input file</strong></summary>
    <pre><code class="language-bash">
optics
bse
bsk
bss
ppa
NLogCPUs=1
BS_CPU="4 4 2"                       # [PARALLEL] CPUs for each role
BS_ROLEs="eh k t"                     # [PARALLEL] CPUs roles (k,eh,t)
BSKmod= "SEX"
BSENGexx= 80.0 Ry
BSENGBlk= 5000 mHa
BSSmod= "s"
BSEmod= "resonant"
RandQpts=10000000                     # [RIM] Number of random q-points in the BZ
RandGvec=400
######
BSSNEig= 100
WRbsWF
BSSSlepcMaxIt=10000000
BSSSlepcMatrix
Lkind = "full"
#####
KfnQPdb =     'E < ./gw/ndb.QP'
% BSEBands
  7 | 10
%
% BSEQptR
  1 | 7 |   #244                        # [BSK] Transferred momenta range
%
%KfnQP_E
2.000000|1.000000|1.000000|             #       (EXTQP)(BSK)(BSS)       E parameters (c/v) eV|adim|adim
%
% BDmRange
  0.0100000 | 0.0100000 | eV
%
% BEnRange
  0.000000 | 8.000000 | eV
%
BEnSteps= 1000
DIP_CPU= "4 2 2"                      # [PARALLEL] CPUs for each role
DIP_ROLEs= "k c v"                    # [PARALLEL] CPUs roles (k,c,v)
% BLongDir
  1.000000 | 1.000000 | 0.000000
%
</code></pre>
</details>

# References

```{bibliography}
    :cited:
```